##### <b>1. Load custom functions</b>

In [14]:
from functions import *
random_state = 1

##### <b>2. Load data and perform EDA (Exploratory Data Analysis)</b>

In [15]:
survey_df = load_data(folder_name="data", file_name="ACME-HappinessSurvey2020.csv")
# print(survey_df.head(), "\n")
# print(survey_df.info(), "\n")
# survey_df.describe()

The data set is simple - only 126 rows and 6 columns, all integer data type with a range between 0 and 5.<br>
<br>
Since the target variable Y is a categorical variable (0 - unhappy, 1 - happy), convert it accordingly in the data frame.


In [16]:
survey_df["Y"] = survey_df["Y"].astype('category').cat.set_categories([0, 1], ordered=True)

Plot histograms to understand the distribution of each column.

In [17]:
# plot_histograms(data=survey_df,
#                 target="Y", target_figsize=(2,2),
#                 dependent_layout=(2,3), dependent_figsize=(8,6))

For the target variable Y, the distribution is slightly uneven, with more happy customers in the survey data.<br>
<b>To deal with this class imbalance problem, SMOTE (synthetic minority oversampling technique) will be used in the next stages.</b><br>
<br>
The distribution of each dependent variable is also not uniform. However, it would not matter too much as long as the variables/features can provide predictive power.<br>
To better understand these features, perform Chi-square tests.

##### <b>3. Feature selection</b>

In [18]:
# chi_independence_df = run_chi_tests(data=survey_df, target="Y", significance_level=0.05,
#                                     plot_row=2, plot_col=3, figsize=(8,5))
# print("----------------------------------------------------------------------------")
# print("----------------------------------------------------------------------------")
# print("2. Chi-square test of independence")
# print("----------------------------------------------------------------------------")
# print("----------------------------------------------------------------------------")
# print("Table 1. Result of Chi-square test of independence (X1-6 and Y)")
# chi_independence_df.set_index("Independent Variable")

<b>Chi-square test of goodness of fit</b><br>
- This statistical test is often used to evaluate whether or not sample data is representative of the full population.
- The null hypothesis was that there is no significant difference between a variable and its expected frequencies.
- As we failed to reject the null hypothesis for all 6 dependent variables, we can consider that they are representative of the population at a significance level (or alpha) of 0.05.

<b>Chi-square test of independence</b><br>
- This statistical test is used to evaluate whether or not a difference between observed expected data is due to chance. We can consider a relationship between the variables exists when failing to reject the null hypothesis.
- The null hypothesis was that a dependent variable X# and the target variable Y are independent of each other.
- At a significance level of 0.05, we were able to reject the null hypothesis only for X1.
- If we were to tolerate a higher chance of error by increasing the alpha to 0.1, we would be able to reject the null hypothesis for X6 as well, which would mean that X6 and Y are not independent of each other.
- In summary, X2-X5 are independent of Y, meaning they would not be helpful in predicting Y. Whereas X1 would be helpful in predicting Y at a significance level of 0.05 and X6 as well, at a significance level of 0.1.

<b>Relationship between target Y and each dependent variable</b><br>
- X1 and X5 seem to be good features for training a predictive model based on the line charts above - higher X values (dependent variable) generally correspond to higher Y values (target variable). However, the chi-square test of independence failed to reject the null hypothesis of "X5 and Y are independent of each other" when alpha=0.05.
- On the contrary, it appears that it would be hard to predict Y based on the other X variables as fluctuations are observed from the line charts, which were confirmed by the chi-square test of independence.

<b>Summary and next step</b><br>
- <b>X1 (my order was delivered on time) appears to be the most relevant feature to the target Y (customer satisfaction).</b>
- However, to build a more robust model, it would be reasonable to use at least 2 features rather than discarding 5 out of 6 features, especially because the data are not complex nor big.
- X5 and X6 could be useful in training a predictive model but it is unclear at this stage whether using either or both of them would be better.
- <b>As such, perform the chi-square test of independence again</b>, but this time to test whether there is a relationship between X1 and the other dependent variables <b>to determine what features other than X1 can be useful in model training.</b>

In [19]:
# chi_independence_df_X1 = run_chi_tests(data=survey_df.drop(["Y"], axis=1), target="X1", significance_level=0.05,
#                                        plot_row=2, plot_col=3, figsize=(8,5), plot=False,
#                                        goodness_of_fit_test=False)
# print("Table 2. Result of Chi-square test of independence (X1 and X2-6)")
# chi_independence_df_X1.set_index("Independent Variable")

X3, X5 and X6 were found to be not independent of X1, meaning they are related to X1.<br>
<br>
With that, we can now try different combinations of the features (i.e. X1, X3, X5 and X6) to see which combination would result in the best prediction accuracy score.<br>
<br>
Before doing so, evaludate different classifiers to choose the base model for the next steps.

##### <b>4. Model selection</b>

First, split the data into train and test.<br>
<br>
As stated above, SMOTE (synthetic minority oversampling technique) will be used for train and test data separately, to handle the class imbalance problem of target Y.<br>
<br>
Train data will be used for model selection, feature engineering, and hyperparameter tuning.<br>
Test data will only be used at the last step to evaluate the fine-tuned model.<br>

In [20]:
X_train, X_test, y_train, y_test = split_data(
    X=survey_df.drop(["Y"], axis=1),
    y=survey_df["Y"],
    test_size=0.2,
    random_state=random_state,
    oversampling=True)

In [21]:
# eval_models(X=X_train, y=y_train, n_splits=5, n=100, random_state=random_state)

Tried some of the most common classifiers.
# RandomForestClassifier sometimes outperformed XGBClassifier but XGBClassifier was so much faster and the performance was as good as RandomForestClassifier.
(add description of XGB classifier)
https://www.nvidia.com/en-us/glossary/data-science/xgboost/

Use this classifier as our base model for the later steps. Moving forward, try different combinations of the features (X1, X3, X5 and X6) to see which combination would result in the best prediction accuracy score.

In [22]:
# eval_feature_combinations(X=X_train, y=y_train, n_splits=5, n=100, random_state=random_state,
#                           classifier=XGBClassifier(verbosity=0, random_state=random_state))

<b>X1 and X3</b> resulted in the best mean prediction accuracy score, thus the other features will not be used from now on.<br>
<br>
Re-define X_train and X_test with the best features combination for the later steps.


In [23]:
X_train = X_train[["X1", "X3"]]
X_test = X_test[["X1", "X3"]]

##### <b>5. Feature augmentation</b>

With the three features, try different data augmentation/transformation techniques that will create new features out of the existing features to see whether they improve prediction accuracy.

In [24]:
# eval_transformers(X=X_train, y=y_train, n_splits=5, n=100, random_state=random_state,
#                   classifier=XGBClassifier(verbosity=0, random_state=random_state))

# the best was PolynomialCountSketch -> what it is and why it performs well with the data we have?

As we found the mean accuracy scores improved with the transformers, use the best performing transformer (i.e. PolynomialCountSketch) for the next steps.

In [25]:
transformer=PolynomialCountSketch(random_state=random_state)
X_train = transformer.fit_transform(X_train)
X_test = transformer.transform(X_test)

##### <b>6. Dimensionality reduction</b>
Now we have more features due to the feature augmentation process, try different dimensionality reduction techniques to see if they help improve accuracy score.

In [26]:
eval_decomposers(X=X_train, y=y_train, n_splits=5, n=100, random_state=random_state,
                classifier=XGBClassifier(verbosity=0, random_state=random_state))

Classifier: PCA(random_state=1)
{'Mean': 0.5336, 'Std': 0.1735, 'Max': 0.938, 'Min': 0.0909, 'Time elapsed': '00:00:52'} 

Classifier: KernelPCA(random_state=1)
{'Mean': 0.7564, 'Std': 0.0931, 'Max': 0.9876, 'Min': 0.4008, 'Time elapsed': '00:00:37'} 

Classifier: FastICA(random_state=1)
{'Mean': 0.6889, 'Std': 0.1215, 'Max': 0.9711, 'Min': 0.2438, 'Time elapsed': '00:16:40'} 

Classifier: SparsePCA(random_state=1)
{'Mean': 0.7385, 'Std': 0.096, 'Max': 0.9876, 'Min': 0.3967, 'Time elapsed': '00:08:27'} 

Classifier: IncrementalPCA()
{'Mean': 0.7359, 'Std': 0.1012, 'Max': 0.9876, 'Min': 0.3884, 'Time elapsed': '00:00:44'} 

Classifier: TruncatedSVD()
{'Mean': 0.7452, 'Std': 0.0958, 'Max': 0.9876, 'Min': 0.3884, 'Time elapsed': '00:00:27'} 



Classifier: PCA(random_state=1)
{'Mean': 0.5336, 'Std': 0.1735, 'Max': 0.938, 'Min': 0.0909, 'Time elapsed': '00:00:52'} 

Classifier: KernelPCA(random_state=1)
{'Mean': 0.7564, 'Std': 0.0931, 'Max': 0.9876, 'Min': 0.4008, 'Time elapsed': '00:00:37'} 

Classifier: FastICA(random_state=1)
{'Mean': 0.6889, 'Std': 0.1215, 'Max': 0.9711, 'Min': 0.2438, 'Time elapsed': '00:16:40'} 

Classifier: SparsePCA(random_state=1)
{'Mean': 0.7385, 'Std': 0.096, 'Max': 0.9876, 'Min': 0.3967, 'Time elapsed': '00:08:27'} 

Classifier: IncrementalPCA()
{'Mean': 0.7359, 'Std': 0.1012, 'Max': 0.9876, 'Min': 0.3884, 'Time elapsed': '00:00:44'} 

Classifier: TruncatedSVD()
{'Mean': 0.7452, 'Std': 0.0958, 'Max': 0.9876, 'Min': 0.3884, 'Time elapsed': '00:00:27'} 

In [ ]:
# KernelPCA performed the best?
decomposer=KernelPCA(random_state=random_state)
X_train = decomposer.fit_transform(X_train)
X_test = decomposer.transform(X_test)

Some decomposers performed better than the others but none of them significantly improved the accuracy score.<br>
Hence, proceed without a dimensionality reduction step.<br>
<br>
Next is hyperparameter tuning for the classifier.

##### <b>7. Hyperparameter tuning</b>

In [ ]:
transformers = [
    PolynomialCountSketch(random_state=random_state, n_components=n) for n in np.arange(50, 350, 50)
    ]
eval_transformers(X=X_train, y=y_train, n_splits=5, n=100, random_state=random_state,
                  classifier=XGBClassifier(verbosity=0, random_state=random_state))

transformers = [
    PolynomialCountSketch(skewedness=random.choice(np.arange(0.01, 1, 0.01))) for _ in range(10)
    ]
eval_transformers(X=X_train, y=y_train, n_splits=5, n=100, random_state=random_state,
                  classifier=XGBClassifier(verbosity=0, random_state=random_state))

In [ ]:
decomposers = [

]
eval_decomposers(X=X_train, y=y_train, n_splits=5, n=100, random_state=random_state,
                classifier=XGBClassifier(verbosity=0, random_state=random_state))

decomposers = [
    
]
eval_decomposers(X=X_train, y=y_train, n_splits=5, n=100, random_state=random_state,
                classifier=XGBClassifier(verbosity=0, random_state=random_state))

In [15]:
best_search = tune_xgb_clr(X=X_train, y=y_train, n_iter=200, random_state=random_state)

XGBClassifier(alpha=0.1, base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eta=0.27, eval_metric=None,
              feature_types=None, gamma=None, gpu_id=None, grow_policy=None,
              importance_type=None, interaction_constraints=None, lambda=0.03,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=9,
              max_leaves=None, min_child_weight=3, missing=nan,
              monotone_constraints=None, n_estimators=100, n_jobs=None, ...) 
 0.7 
 {'tree_method': 'approx', 'subsample': 0.30000000000000004, 'min_child_weight': 3, 'max_depth': 9, 'lambda': 0.03, 'eta': 0.27, 'alpha': 0.1}


0.5

##### <b>8. Evaluate the tuned model using test data</b>

In [ ]:
scores = cross_val_score(best_search.best_estimator_, X_test, y_test, scoring='roc_auc',
                         cv=RepeatedStratifiedKFold(n_splits=5, n_repeats=100, random_state=random_state))
np.mean(scores)

##### <b>9. Save the best performing model for future use</b>

##### <b>10. Conclusion</b>

learning points, insights, future work, etc.